In [2]:
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
import plotly
import plotly.graph_objects as go

# Obtaining meshgrids for graphing

In [5]:
water_file = nc.Dataset('data/pottmp.1998.nc')
water_lons = wetfile.variables['lon'][:]
water_lats = wetfile.variables['lat'][:]

air_file = nc.Dataset('data/air.mon.meanv3.nc')
air_lons = dryfile.variables['lon'][:]
air_lats = dryfile.variables['lat'][:]
pressure = dryfile.variables['level'][:]

In [ ]:
air_size = air_file.variables['air'][0].size
layer = air_file.variables['air'][0,0].size
airheights = np.load('January airheights.npy')
xxair, yyair = np.meshgrid(air_lons, air_lats)
xxwater, yywater = np.meshgrid(water_lons, water_lats)

In [ ]:
A= np.reshape(airheights[0:181*360], (181, 360))

# Getting File and clipping max and min values

In [ ]:
#This is just for January. Also works for other months respectively
EOFs = np.load('January Physical EOFs.npy')

In [ ]:
#For this example, we are graphing January EOF Mode 1. 
#For different modes, change 0 to (mode # -1)
lim = np.minimum(np.abs(np.nanmin(EOFs[:,0])), np.nanmax(EOFs[:,0]))
PhysicalEOFs = np.clip(EOFs[:,0], -lim, lim)

In [ ]:
#Done to make values extreme as Plotly reads NaNs as 0's when graphing.
for j in range(0, PhysicalEOFs.shape[0]):
    if np.isnan(PhysicalEOFs[j,0]):
        PhysicalEOFs[j,0] = 1

# Creating Custom Colorscale for Figure

In [ ]:
#Number of colors, choice color, etc can be adjusted. 
colors = [[0, "black"],[0.01, "violet"],[1/6+0.05, "blue"],[1/3, "cyan"],
          [1/2, "white"],[2/3, "yellow"],[5/6, "orange"],[1, "red"]]

# Graphing and Exporting 3D Figure

In [ ]:
# 0 minus because vectors parity. This must be checked manually when graphing
fig = go.Figure(data=[
    go.Surface(x = xxair, y= yyair, z=A, 
               surfacecolor = 0-np.reshape(PhysicalEOFs[0:181*360*1,0],xxair.shape),
               colorscale = colors,  opacity = 1),
])
for i in range(0,25):
#Flexible to however many layers wanted. This is for 25 layers up
    B = np.reshape(airheights[i*layer:(i+1)*layer], xxair.shape)
    fig.add_trace(go.Surface(x =xxair, y =yyair, z=B,
                             surfacecolor = 0-np.reshape(PhysicalEOFs[layer*i:layer*(i+1),0],xxair.shape),
                             colorscale = 'Edge', showscale = False, opacity = 0.9))

fig.add_trace(go.Surface(x =xxwater, y =yywater, z=np.ones(xxwater.shape)*-5,
                        surfacecolor = 0-np.reshape(PhysicalEOFs[air_size:,0],xxwater.shape),
                         colorscale = colors, showscale = False, opacity = 1))

fig.update_traces(cmax=lim, selector=dict(type='surface'))
fig.update_traces(cmin=-lim, selector=dict(type='surface'))

#Colorbar formatting. Can change to one's liking
fig.update_traces(colorbar_exponentformat="power", colorbar_showexponent= "none")
fig.update_traces(colorbar_tickfont_size=20)
fig.update_traces(colorbar_thickness=45)
fig.update_traces(colorbar=dict(title=dict(text=r'x 10^{-5}'), xpad = 50))
fig.update_traces(colorbar_title_font_size=15)


fig.update_layout(title = 'January Physical EOFs Mode 1',
                 scene = dict(aspectratio=dict(x=1.75,y=1,z=1.5)))
fig.update_layout(scene = dict(
                xaxis_title='Longitude',
                yaxis_title='Latitude',
                zaxis_title='Meters'))
fig.write_html("Jan 3D Physical EOFs (5m down).html")